In [4]:
from bs4 import BeautifulSoup
import pandas as pd

# Загружаем HTML файл
with open("hockey.html", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# Подготовим список для данных
data = []

# Парсинг первой части
matches_part1 = soup.find_all("div", {"id": "match", "class": "item"})

for match in matches_part1:
    date = match.find("span", id=lambda x: x and "DL" in x).text.strip().split(' ')[1]
    time = match.find("span", id=lambda x: x and "TL" in x).text.strip()
    teams = match.find_all("a", id=lambda x: x and "THL" in x)
    scores = match.find_all("a", id=lambda x: x and "SHL" in x)

    home_team = teams[0].text.strip()
    away_team = teams[1].text.strip()
    home_score = scores[0].text.strip().replace('ТР', '').replace('ПБ', '')
    away_score = scores[1].text.strip().replace('ТР', '').replace('ПБ', '')
    # Добавляем данные в общий список
    if home_team == 'Черные Медведи Политех':
        data.append([date, time, home_team, away_team, home_score, away_score])
    else:
        data.append([date, time,away_team,  home_team,  away_score, home_score])


# Парсинг второй части
matches_part2 = soup.find("table", {"id": "MatchGridView"}).find_all("tr")[2:]

for row in matches_part2:
    cells = row.find_all("td")
    if len(cells) < 7:  # Пропускаем строки без данных
        continue

    date = cells[2].text.strip().split(' ')[1]
    time = cells[3].text.strip()
    teams = cells[5].find("a").text.strip().split(" - ")
    result = cells[6].text.strip().split(" : ")

    home_team = teams[0].strip()
    away_team = teams[1].strip()
    home_score = result[0].strip().replace('ТР', '').replace('ПБ', '')
    away_score = result[1].strip().replace('ТР', '').replace('ПБ', '')

    # Добавляем данные в общий список
    if home_team == 'Черные Медведи Политех':
        data.append([date, time, home_team, away_team, home_score, away_score])
    else:
        data.append([date, time,away_team,  home_team,  away_score, home_score])

# Создаем DataFrame
columns = ["date", "time", "team1", "team2", "score1", "score2"]
df = pd.DataFrame(data, columns=columns)
df = df[(df.score1!='') & (df.score2!='')]
df['score1'] = df['score1'].astype(int)
df['score2'] = df['score2'].astype(int)
# Сохраняем итоговый DataFrame


print("Данные успешно объединены и сохранены")


Данные успешно объединены и сохранены


In [5]:
df.to_csv('hockey.csv', index=False)

In [7]:
# model.py

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class Model:
    def __init__(self):
        model_name = "Qwen/Qwen2.5-0.5B-Instruct"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else {"": self.device}
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.system_prompt = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."
        self.task_prefix = "Your task is to analyes table:"
    
    def predict(self, user_prompt):
        full_prompt = f"{self.task_prefix} {user_prompt}"
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": full_prompt}
        ]
        
        # Применяем шаблон чата, если метод доступен
        if hasattr(self.tokenizer, "apply_chat_template"):
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
        else:
            # Альтернативный способ формирования запроса
            text = f"{self.system_prompt}\n{full_prompt}"
        
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)
        
        generated_ids = self.model.generate(**model_inputs,max_new_tokens=512)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

# Пример использования модели при запуске model.py напрямую
if __name__ == "__main__":
    model = Model()
    user_input = input("Введите ваш запрос: ")
    response = model.predict(user_input)
    print("Ответ модели:")
    print(response)


model.safetensors:   2%|2         | 21.0M/988M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
"7592943669:AAEtdkKW1wQcXK2vwrRbO5SpahciPHBJlnQ"

In [ ]:
import pandas as pd
df = pd.read_csv(f"hockey.csv")
df = df.sort_values(by=['date'], ascending=False)[:5]
ans = f""
for i in range(len(df)):
    row = df.iloc[i]
    ans += f"{row['date']} {row['time']} {row['score1']}:{row['score2']}\n"

In [13]:
df = pd.read_csv(f"hockey.csv")
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['date'], ascending=False)[:5]

C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_824\725270472.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


,date,time,team2,score1,score2
8,2024-11-14,19:00,Невские Львы,4,3
7,2024-11-08,19:00,Невские Львы,5,2
6,2024-10-31,19:00,Северные Сапсаны,3,2
5,2024-10-25,19:00,Северные Сапсаны,5,1
4,2024-10-18,19:00,Динамо СПБГЭУ,6,4
